<a href="https://colab.research.google.com/github/pazlshet/balenaos-helium-gtw/blob/master/Copy_of_YT_Prompt_Wizard_GSM8k_GPT_4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptWizard GSM8k

In [ ]:
!git clone https://github.com/microsoft/PromptWizard

Cloning into 'PromptWizard'...
remote: Enumerating objects: 549, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 549 (delta 59), reused 58 (delta 58), pack-reused 486 (from 1)
Receiving objects: 100% (549/549), 1.67 MiB | 13.29 MiB/s, done.
Resolving deltas: 100% (280/280), done.


In [ ]:
cd PromptWizard

/content/PromptWizard


In [ ]:
ls

CODE_OF_CONDUCT.md  docs/    LICENSE   promptwizard/   README.md          SECURITY.md  setup.py
demos/              images/  Makefile  pyproject.toml  RESPONSIBLE_AI.md  setup.cfg


In [ ]:
!pip install -e .

Obtaining file:///content/PromptWizard
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.0 MB/s eta 

In [ ]:
import openai
openai.__version__

'1.59.3'

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
# os.environ["OPENAI_API_KEY"]
os.environ["USE_OPENAI_API_KEY"] = "True"
os.environ["OPENAI_MODEL_NAME"] ="gpt-4o"
os.environ["OPENAI_API_VERSION"] = "1.59.3"

### Import Dependencies

In [ ]:
import sys
sys.path.insert(0, "../../")
import promptwizard
from promptwizard.glue.promptopt.instantiate import GluePromptOpt
from promptwizard.glue.promptopt.techniques.common_logic import DatasetSpecificProcessing
from promptwizard.glue.common.utils.file import save_jsonlist
from typing import Any
from tqdm import tqdm
from re import compile, findall
import os
from datasets import load_dataset

# from dotenv import load_dotenv
# load_dotenv(override = True)

### Create a dataset specific class and define the required functions

In [ ]:
class GSM8k(DatasetSpecificProcessing):

    def dataset_to_jsonl(self, dataset_jsonl: str, **kwargs: Any) -> None:
        def extract_answer_from_output(completion):
            # Your functions for metrics and prompt building
            ans_re = compile(r"#### (\-?[0-9\.\,]+)")
            self.INVALID_ANS = "[invalid]"

            match = ans_re.search(completion)
            if match:
                match_str = match.group(1).strip()
                match_str = match_str.replace(",", "")
                return match_str
            else:
                return self.INVALID_ANS

        examples_set = []

        for _, sample in tqdm(enumerate(kwargs["dataset"]), desc="Evaluating samples"):
            example = {
              DatasetSpecificProcessing.QUESTION_LITERAL: sample['question'],
              DatasetSpecificProcessing.ANSWER_WITH_REASON_LITERAL: sample['answer'],
              DatasetSpecificProcessing.FINAL_ANSWER_LITERAL: extract_answer_from_output(sample["answer"])
            }
            examples_set.append(example)

        save_jsonlist(dataset_jsonl, examples_set, "w")

    def extract_final_answer(self, answer: str):

        if not answer:
            return self.INVALID_ANS

        model_pred = answer.lower()
        preds = model_pred.split(self.ANSWER_START.lower())
        answer_flag = True if len(preds) > 1 else False

        pred = preds[-1].replace(",", "")
        pred = [s for s in findall(r'-?\d+\.?\d*', pred)]

        if len(pred) == 0:
            return self.INVALID_ANS

        if answer_flag:
            # choose the first element in list
            pred = pred[0]
        else:
            # choose the last element in list
            pred = pred[-1]

        # (For arithmetic tasks) if a word ends with period, it will be omitted ...
        if pred[-1] == ".":
            pred = pred[:-1]
        return pred

In [ ]:
gsm8k_processor = GSM8k()

### Load and save the dataset

In [ ]:
if not os.path.exists("data"):
    os.mkdir("data")

dataset = load_dataset("openai/gsm8k", "main")
num_samples = 0
for dataset_type in ['train','test']:
    data_list = []
    for data in dataset[dataset_type]:
        data_list.append({"question": data['question'], "answer": data['answer']})
        if num_samples == 100 and dataset_type == 'train': # We sample only 100 train examples and use 25 out them for training randomly
            break
        num_samples += 1
    gsm8k_processor.dataset_to_jsonl("data/"+ dataset_type+'.jsonl', dataset=data_list)

Evaluating samples: 101it [00:00, 221445.22it/s]
Evaluating samples: 1319it [00:00, 264323.31it/s]


### Set paths

In [ ]:
ls demos/gsm8k

configs/  demo.ipynb


In [ ]:
train_file_name = os.path.join("data", "train.jsonl")
test_file_name = os.path.join("data", "test.jsonl")
path_to_config = "demos/gsm8k/configs"
promptopt_config_path = os.path.join(path_to_config, "promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

In [ ]:
!cat {setup_config_path}

assistant_llm:
  # put the unique_model_id that you specified in llm_config.yaml
  prompt_opt: gpt-4o
dir_info:
  # Base directory for everything
  base_dir: logs
  log_dir_name: glue_logs
experiment_name: gsm8k
# Many features are different for mode: online/offline. For eg
# 1) Print of logs happens on console for offline mode
# 2) LLM Queue gets instantiated only in online mode
mode: offline
# Full length description of the experiment. This would be logged.
description:


In [ ]:
!cat {promptopt_config_path}

# Specify one or more prompt refinement technique to be used. If you specify more than one prompt refinement techniques,
# all these technique would run on same seed data. Result, iterations needed & cost incurred for each of these
# technique would be logged. And winning technique for each data instance and overall would be logged.

# Supported prompt refinement techniques: Basic, RecursiveEval, MedPrompt
# Uncomment techniques that you want to use
############################ Critique Task Description Start ############################
prompt_technique_name: "critique_n_refine"
# unique_model_id of model defined in llm_config.yaml
unique_model_id: gpt-4o
# Number of iterations for conducting <mutation_rounds>  rounds of mutation of task description
# followed by refinement of instructions
mutate_refine_iterations: 3
# Number of rounds of mutation to be performed when generating different styles
mutation_rounds: 3
# Refine instruction post mutation
refine_instruction: true
# Number of

### Create an object for calling prompt optimization and inference functionalities

In [ ]:
gp = GluePromptOpt(promptopt_config_path,
                   setup_config_path,
                   train_file_name,
                   gsm8k_processor)

Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'gsm8k'), ('mode', 'offline')] 


Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'gsm8k'), ('mode', 'offline')] 


Prompt Optimization parameters: [('answer_format', 'For each question present the reasoning followed by the correct answer.'), ('base_instruction', 'Lets think step by step.'), ('few_shot_count', 5), ('generate_expert_identity', True), ('generate_intent_keywords', False), ('generate_reasoning', True), ('max_eval_batches', 6), ('min_correct_count', 3), ('mutate_refine_iterations', 3), ('mutation_rounds', 3), ('num_train_examples', 20), ('prompt_technique_name', 'critique_n_refine'), ('questions_batch_size', 1), ('refine_instruction', True), 

### Call prompt optmization function
1. ```use_examples``` can be used when there are training samples and a mixture of real and synthetic in-context examples are required in the final prompt. When set to ```False``` all the in-context examples will be real
2. ```generate_synthetic_examples``` can be used when there are no training samples and we want to generate synthetic examples
3. ```run_without_train_examples``` can be used when there are no training samples and in-context examples are not required in the final prompt

In [ ]:
# Function call to generate optimal prompt and expert profile
best_prompt, expert_profile = gp.get_best_prompt(use_examples=True,run_without_train_examples=False,generate_synthetic_examples=False)


Mutating Task Description....


Iterations completed:   0%|          | 0/3 [00:00<?, ?it/s]
 + Starting iteration: 1 
 current_base_instruction: Lets think step by step.

 + Starting iteration: 1 
 current_base_instruction: Lets think step by step.
mutation_round=0 mutated_sample_prompt=You are given a task description and a prompt instruction and different styles known as meta prompts:
[Task Description]: You are a mathematics expert. You will be given a mathematics problem which you need to solve
[Meta Prompt]: How could I devise an experiment to help solve that problem?
Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.
How could I measure progress on this problem?
How can I simplify the problem so that it is easier to solve?
What are the key assumptions underlying this problem?
Now you need to generate 5 variations of following Instruction adaptively mixing meta prompt while keeping similar semantic meaning.
Make sure to wrap each generated 

dataset_subset [{'question': 'Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?', 'answer': 'In one hour, there are 3 sets of 20 minutes.\nSo, Joy can read 8 x 3 = <<8*3=24>>24 pages in an hour.\nIt will take her 120/24 = <<120/24=5>>5 hours to read 120 pages.\n#### 5', 'final_answer': '5'}]
critique_example_set, correct_count
[] 1
dataset_subset [{'question': 'It takes Roque two hours to walk to work and one hour to ride his bike to work. Roque walks to and from work three times a week and rides his bike to and from work twice a week. How many hours in total does he take to get to and from work a week with walking and biking?', 'answer': 'Roque takes 2*3 = <<2*3=6>>6 hours a week to walk to work.\nRoque takes 6*2 = <<6*2=12>>12 hours a week to walk to and from work.\nRoque takes 1*2 = <<1*2=2>>2 hours a week to bike to work.\nRoque takes 2*2 = <<2*2=4>>4 hours a week to bike to and from work.\nIn total, Roque takes 12+4 = <<12+4=16>>16 ho

prompt_score_list [['You are a mathematics expert. You will be given a mathematics problem which you need to solve\nLets think step by step.', 1.0, [{'question': 'Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?', 'answer': 'To the initial 2 pounds of jelly beans, he added enough brownies to cause the weight to triple, bringing the weight to 2*3=<<2*3=6>>6 pounds.\nNext, he added another 2 pounds of jelly beans, bringing the weight to 6+2=<<6+2=8>>8 pounds.\nAnd finally, he added enough gummy worms to double the weight once again, to a final weight of 8*2=<<8*2=16>>16 pounds.\n#### 16', 'final_an

dataset_subset [{'question': 'A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?', 'answer': 'Let S be the number of people on the first hundred years’ ship.\nThe second hundred years’ ship had twice as many as the first, so it had 2S people.\nThe third hundred years’ ship had twice as many as the second, so it had 2 * 2S = <<2*2=4>>4S people.\nAll the ships had S + 2S + 4S = 7S = 847 people.\nThus, the ship that the monster ate in the first hundred years had S = 847 / 7 = <<847/7=121>>121 people on it.\n#### 121', 'final_answer': '121'}]
critique_example_set, correct_count
[] 3
Loop completed


...

100%|██████████| 3/3 [00:50<00:00, 16.68s/it]



Generating CoT Reasoning for In-Context Examples....


100%|██████████| 5/5 [00:41<00:00,  8.25s/it]



Generating Expert Identity....


Expert Identity: You are a mathematics expert with a deep understanding of various mathematical disciplines, including algebra, calculus, geometry, and statistics. You have a strong analytical mind and are skilled at breaking down complex problems into manageable steps. Your expertise allows you to approach mathematical problems with precision and creativity, finding efficient and accurate solutions. You are familiar with a wide range of mathematical techniques and tools, and you can apply them effectively to solve problems. Whether the problem involves solving equations, analyzing data, or proving theorems, you can provide clear and logical solutions. Your ability to communicate complex mathematical concepts in an understandable way makes you an invaluable resource for anyone seeking to understand or solve mathematical problems.
Expert Identity: You are a mathematics expert with a deep understanding of various mathematical disciplines, including algebra, calculus, geometry, and statis

## Show the Best

In [ ]:
# Best Prompt
best_prompt

"\n**Enhanced Prompt for Solving Mathematical Problems**\n\nYou are a mathematics expert tasked with solving a variety of mathematical problems. Follow this structured approach to ensure clarity, accuracy, and logical reasoning:\n\n1. **Comprehend the Problem**: Thoroughly read the problem statement to understand the context and determine what is being asked. Pay close attention to all details, including units and context, to ensure a comprehensive understanding.\n\n2. **Extract Key Information**: Identify and list all important data, variables, and conditions provided in the problem. Ensure no critical information is overlooked, especially in problems involving costs, measurements, or percentages.\n\n3. **Plan Strategically**: Develop a logical plan by breaking down the problem into manageable steps. Consider different methods and choose the most suitable approach. Pay special attention to the sequence of arithmetic operations and ensure unit consistency throughout.\n\n4. **Execute wi

In [ ]:
print(f"Best prompt: {best_prompt} \nExpert profile: {expert_profile}")

Best prompt: 
**Enhanced Prompt for Solving Mathematical Problems**

You are a mathematics expert tasked with solving a variety of mathematical problems. Follow this structured approach to ensure clarity, accuracy, and logical reasoning:

1. **Comprehend the Problem**: Thoroughly read the problem statement to understand the context and determine what is being asked. Pay close attention to all details, including units and context, to ensure a comprehensive understanding.

2. **Extract Key Information**: Identify and list all important data, variables, and conditions provided in the problem. Ensure no critical information is overlooked, especially in problems involving costs, measurements, or percentages.

3. **Plan Strategically**: Develop a logical plan by breaking down the problem into manageable steps. Consider different methods and choose the most suitable approach. Pay special attention to the sequence of arithmetic operations and ensure unit consistency throughout.

4. **Execute w

### Save the optimized prompt and expert profile

In [ ]:
import pickle

if not os.path.exists("results"):
    os.system("mkdir results")

with open("results/best_prompt.pkl", 'wb') as f:
    pickle.dump(best_prompt, f)
with open("results/expert_profile.pkl", 'wb') as f:
    pickle.dump(expert_profile, f)

print(f"Best prompt: {best_prompt} \nExpert profile: {expert_profile}")

Best prompt: 
**Enhanced Prompt for Solving Mathematical Problems**

You are a mathematics expert tasked with solving a variety of mathematical problems. Follow this structured approach to ensure clarity, accuracy, and logical reasoning:

1. **Comprehend the Problem**: Thoroughly read the problem statement to understand the context and determine what is being asked. Pay close attention to all details, including units and context, to ensure a comprehensive understanding.

2. **Extract Key Information**: Identify and list all important data, variables, and conditions provided in the problem. Ensure no critical information is overlooked, especially in problems involving costs, measurements, or percentages.

3. **Plan Strategically**: Develop a logical plan by breaking down the problem into manageable steps. Consider different methods and choose the most suitable approach. Pay special attention to the sequence of arithmetic operations and ensure unit consistency throughout.

4. **Execute w

### Evaluate the optimized prompt

In [ ]:
gp.EXPERT_PROFILE = expert_profile
gp.BEST_PROMPT = best_prompt

# Function call to evaluate the prompt
accuracy = gp.evaluate(test_file_name)

print(f"Final Accuracy: {accuracy}")

Streaming output truncated to the last 5000 lines.
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/e

Final Accuracy: 0.9438968915845337
